In [552]:
# NOTES

# Mind map
# https://miro.com/app/board/o9J_kwo5k8s=/

# Prediction labels
# [       0     ,      1    ,   2  ]
# ["pedestrian" , "bicycle" , "car"]

# Coordinate format: [longitude, latitude]

# Time format used:
# "%Y-%m-%dT%H:%M:%S.%fZ"

# Sequence format: Geojson
#     type
#     properties:
#       captured_at (start of sequence)
#       coordinateProperties
#         cas (camera angle between 0 and 360)
#         image_keys
#     geometry
#       coordinates [longitude, latitude]

# Image format: Geojson
#    type
#    properties:
#       captured_at, 
#       camera_model, camera_make, ca, pano, seq_key, key, username, user_key
#    geometry
    

# Potential features
#     Average velocity
#     Maximum velocity
#     Maximum stop time (calculate time between neighbours points relative to avg speed)
#     Possible to use CAS coordinate property to estimte slope and the difference in velocity?
#     Smallest turn (small turn will indicate not a car road)
#     Trajectory measurements
#     length
#     
#     
#     Image detection
#       Traffic lights (number or boolean)
#       Roundabouts (number or boolean)
#       Bicycle tracks on the side
#       Highway/number or lanes
#       Sidewalk

# Features columns
# 0 =label, 1 = seq_id, 2 = avg_speed(m/s) 

# Imports

In [553]:
# Imports
import requests
import json
import math
import numpy as np
import random
import csv
import pandas
from datetime import datetime, date
from pprint import pprint as pp
from keras.models import Sequential
from keras.layers.core import Dense, Flatten, Dropout
from keras.layers.convolutional import Conv1D
from keras.layers.convolutional import MaxPooling1D
from keras.optimizers import SGD
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split

# API requests to Mapillary

In [554]:
# Mapillary
def requestImageById(image_key):
    request = requests.get("https://a.mapillary.com/v3/images/"+image_key+"?client_id="+client_id)
    return request.json()

def requestSequencesByBbox(ul, lr):
    # i prefer [lat, lng], so have to keep that in mind
    # request of bbox is box=minx,miny,maxx,maxy=ul_lng, lr_lat, lr_lng, ul_lat
    request = requests.get(f"https://a.mapillary.com/v3/sequences?bbox={ul[1]},{lr[0]},{lr[1]},{ul[0]}&per_page=10000&client_id={client_id}")
    return request.json()

def requestSequenceById(sequence_key):
    request = requests.get("https://a.mapillary.com/v3/sequences/"+sequence_key+"?client_id="+client_id)
    return request.json()

def requestImagesBySequence(sequence_key):
    request = requests.get("https://a.mapillary.com/v3/images/?sequence_keys="+sequence_key+"&per_page=10000&client_id="+client_id)
    return request.json()

# Initial test data

In [555]:
# Initial requests from API url (23.08.19)
config = readJson("config.json")
client_id=config["client_id"]

# Current problem: only retrieves max 1000..
area_data =requestSequencesByBbox(config["test_area"][0], config["test_area"][1])


# Utility functions

In [556]:
# Utility functions
def subtractTime(end, start):
    return 0

def distance(end, start):
    end = [deg2rad(end[0]),deg2rad(end[1])]
    start = [deg2rad(start[0]),deg2rad(start[1])]
    
    R =  6373000
    lon1, lat1, lon2, lat2 = start[0], start[1], end[0], end[1]
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = (math.sin(dlat/2)**2)+math.cos(lat1)*math.cos(lat2)*math.sin(dlon/2)**2
    c = 2*math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R*c

def deg2rad(deg):
    return deg*(math.pi/180)

def readJson(file):
    with open(file,"r") as f:
        file=json.load(f)
    return file

def printProgressBar (iteration, total, prefix = '', suffix = '', decimals = 1, length = 100, fill = '█'):
    """
    Call in a loop to create terminal progress bar
    @params:
        iteration   - Required  : current iteration (Int)
        total       - Required  : total iterations (Int)
        prefix      - Optional  : prefix string (Str)
        suffix      - Optional  : suffix string (Str)
        decimals    - Optional  : positive number of decimals in percent complete (Int)
        length      - Optional  : character length of bar (Int)
        fill        - Optional  : bar fill character (Str)
    """
    percent = ("{0:." + str(decimals) + "f}").format(100 * (iteration / float(total)))
    filledLength = int(length * iteration // total)
    bar = fill * filledLength + '-' * (length - filledLength)
    print('\r%s |%s| %s%% %s' % (prefix, bar, percent, suffix), end = '\r')
    # Print New Line on Complete
    if iteration == total: 
        print()



# Feature functions

In [557]:
# Functions
def avg_speed_n_forward(sequence, start,n):
    return 0

# def getP2PFeatures(sequence):
#     prev

def seq_iteration_operations(points):
    prev = 0
    dist = 0
    time = 0
    max_speed = 0
    for curr in range(1,len(points)):
        curr_dist = distance(points[curr]["geometry"]["coordinates"], points[prev]["geometry"]["coordinates"])
        curr_time = datetime.strptime(points[curr]["properties"]["captured_at"], "%Y-%m-%dT%H:%M:%S.%fZ")
        prev_time = datetime.strptime(points[prev]["properties"]["captured_at"], "%Y-%m-%dT%H:%M:%S.%fZ")
        delta_t = (prev_time-curr_time).total_seconds()
        speed_between = 0 if delta_t==0.0 else curr_dist/delta_t
        if(speed_between>max_speed):
            max_speed = speed_between
        dist+=curr_dist
        time+=delta_t
        prev=curr
    avg_speed = 0 if time==0.0 else dist/time
#     print(dist, time, max_speed, avg_speed)
    #in meter
    return (dist, avg_speed, max_speed)

In [558]:
# write features for an area
def area_features(area, write=False, stop_early=False, sample_size=20):
    num_seq = len(area["features"])
    print(f"[INFO] {num_seq} sequences:")
    f = open("features.tsv", "w+")
    for num, seq in enumerate(area_data["features"]):
        if(stop_early==True and num>sample_size):
            break
        printProgressBar(num, num_seq)
        label = 0
        key=seq["properties"]["key"]
        images = requestImagesBySequence(key)["features"]
        num_points = len(images)
        if(num_points < 2):
            continue
#         print(f"Number of points in seq {key}: {num_points}")
        (dist, avg_speed, max_speed) = seq_iteration_operations(images)
        
        if(dist==0 or avg_speed==0):
            continue
        if(avg_speed>5):
            label=2
        elif(avg_speed>1.4):
            label=1
        if(write==True):
            f.write(f"{avg_speed}\t{max_speed}\t{dist}\t{label}\t{key}\n")
    f.close()
    print("[INFO] Done generating features")

# Prepare data

In [635]:
# Structure train and test data
class_names = ["Pedestrian", "Bicycle", "Car"]
n_classes = len(class_names)
def prep_data():
    
    features = np.genfromtxt("features.tsv", delimiter="\t")
    data = features[:,:-2]
    labels = features[:,-2].astype(int)

    (x_train, x_test, y_train, y_test) = train_test_split(data, labels, test_size=0.2, random_state=20)
    
    x_train = x_train.reshape(x_train.shape[0],x_train.shape[1],1)
    x_test = x_test.reshape(x_test.shape[0],x_test.shape[1],1)
    
    lb = LabelBinarizer()
    y_train = lb.fit_transform(y_train)
    y_test = lb.transform(y_test)
    print(x_train.shape, x_test.shape, y_train.shape, y_test.shape) 
    return (x_train, x_test, y_train, y_test)

# Model with Keras

In [643]:
 
def train_model(x_train, x_test, y_train, y_test,epochs=10, lr=0.01):
    model = Sequential()
#     model.add(Dense(36, input_shape=(x_train.shape[1],), activation="sigmoid"))
#     model.add(Dense(18, activation="sigmoid"))
#     model.add(Dense(n_classes, activation="softmax"))
#     model.compile(loss="categorical_crossentropy", optimizer=SGD(lr=lr),metrics=["accuracy"])
#     model.fit(x_train, y_train, validation_data=(x_test, y_test), epochs=epochs, batch_size=1)
    input_shape = (x_train.shape[1],1)
    model.add(Conv1D(filters=64,kernel_size=2,input_shape=input_shape, activation='relu'))
    model.add(Dropout(0.5))
    model.add(MaxPooling1D(pool_size=2))
    model.add(Flatten())
    model.add(Dense(100, activation='relu'))
    model.add(Dense(n_classes, activation='softmax'))
    model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    
    print("[INFO] Training network...")
    model.fit(x_train, y_train, epochs=100, verbose=0)
    print("[INFO] Done training")
    return model

def eval_model(model):
    _, accuracy = model.evaluate(x_test, y_test, verbose=0)
    print("Model accuracy:",accuracy)


In [645]:
# area_features(area_data, write=True, stop_early=False, sample_size=100)
(x_train, x_test, y_train, y_test) = prep_data()
model = train_model(x_train, x_test, y_train, y_test, lr=0.01)
eval_model(model)


(616, 3, 1) (154, 3, 1) (616, 3) (154, 3)
[INFO] Training network...
[INFO] Done training
Model accuracy: 0.7532467516985807
